Test2　　全結合層(linear,relu)のテスト

In [2]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更してね。

In [4]:
%cd "ctorch210824/MiniCTorch_Prototype"

/content/drive/My Drive/Colab Notebooks/ctorch210824/MiniCTorch_Prototype


In [5]:
! pip install lark-parser

     |████████████████████████████████| 229 kB 8.1 MB/s 
  Created wheel for lark-parser: filename=lark_parser-0.11.3-py2.py3-none-any.whl size=99740 sha256=ee1d7806a5997ddb925514ed447368ed7da53ffde7282d9c8479fd9750c4920a
  Stored in directory: /root/.cache/pip/wheels/2d/f0/65/7aea47a49de7acac0108aac16a7ee00eb996f872d978feb87e
Successfully built lark-parser


In [6]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch
import minictorch.generator as GN
import minictorch.converter as CV

In [7]:
class Net( nn.Module ):
   def __init__(self):
      super(Net, self).__init__()
      self.fc1 = nn.Linear(2, 32)
      self.fc2 = nn.Linear(32, 32)
      self.fc3 = nn.Linear(32, 3)

   def forward(self, x):
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x
      #return F.log_softmax(x, dim = 1)


In [8]:
def generate_json( json_path, input_x ): 
   
    model = Net()
    #print("weight", model.fc1.weight)
    #print("bias", model.fc1.bias)
    print(input_x)

    model.eval()
    with torch.no_grad():
        print("[JSON]", json_path )
        GN.generate_minictorch_file( model, input_to_model, json_path )

    return model

In [9]:
def convert_json( project, folder, model, input_x, json_path, rand_flag=0 ):

    #folder = "src"
    cpp_fname   = project + ".cpp"
    param_fname = project + "_param.cpp"
    cpp_path    = folder + "/" + cpp_fname
    param_path  = folder + "/" + param_fname
    make_path   = folder + "/" + "Makefile"

    # load json file
    print( "[JSON]", json_path )
    fp = open( json_path )
    obj = json.load( fp )

    # save parameter file
    code1 = CV.c_param_generator( obj, model, input_x )
    if len(code1) > 0:
       print( "[PARAM]", param_path )
       ofparam = open( param_path, "w" )
       ofparam.write( code1 )
    else:
       param_fname=""

    # save cpp file
    print( "[CPP]  ", cpp_path )
    code2 = CV.c_code_generator( obj, model, rand_flag )

    #print("[SAVE]",args.path+"/"+args.output)
    #ofp= open( args.path + "/" + args.output, "w" )
    ofp = open( cpp_path, "w" )
    ofp.write( code2 )

    # save make file
    print( "[MAKE] ", make_path )
    make_code = CV.makefile_generator( cpp_fname )

    #makefp= open( args.path + "/" + "Makefile", "w" )
    makefp = open( make_path, "w" )
    makefp.write( make_code )


In [10]:
torch.manual_seed( 1 )

input_to_model = torch.randn((1,2),requires_grad=True)

project = "test2"
json_path = 'network/' + project +'.json'

model = generate_json( json_path, input_to_model )

#torch.set_grad_enabled(True)  #　効かない(?)
yy = model( input_to_model )
print(yy)
yy.sum().backward()
print(input_to_model.grad)

tensor([[0.6614, 0.2669]], requires_grad=True)
[JSON] network/test2.json
skip: Net/Linear[fc1]/weight/29
skip: Net/Linear[fc1]/weight/29
skip: Net/Linear[fc2]/weight/32
skip: Net/Linear[fc2]/weight/32
skip: Net/Linear[fc3]/weight/35
skip: Net/Linear[fc3]/weight/35
tensor([[ 0.1536, -0.1925,  0.0064]], grad_fn=<AddmmBackward>)
tensor([[-0.1284,  0.1121]])


In [11]:
 convert_json( project, "src", model, input_to_model, json_path )

[JSON] network/test2.json
{'name': 'Net/Linear[fc1]/weight/43', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [3], 'sorted_id': 1}
{'name': 'Net/Linear[fc1]/bias/42', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [3], 'sorted_id': 2}
{'name': 'Net/Linear[fc2]/weight/46', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [7], 'sorted_id': 5}
{'name': 'Net/Linear[fc2]/bias/45', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [7], 'sorted_id': 6}
{'name': 'Net/Linear[fc3]/weight/49', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [11], 'sorted_id': 9}
{'name': 'Net/Linear[fc3]/bias/48', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [11], 'sorted_id': 10}
[PARAM] src/test2_param.cpp
[CPP]   src/test2.cpp
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'shape': [1, 2], 'out': [3], 'sorted_id': 0}
{'name': 'Net/Linear[fc1]/weight/43', 'op': 'prim::GetAttr', 'in': [], 'shape': [], 'out': [3], 'sorted_id': 1}
Net/Linear[fc1]/weight/43  ->  fc1_weight
{'name': 

In [14]:
!g++ -std=c++14 ./src/test2.cpp ./src/test2_param.cpp -I ../../ctorch/lib -lblas -o ./bin/test2

(注意) ctorch/libにxtensor関連のincludeを置いています。

In [15]:
!./bin/test2

### forward computation ...
{{ 0.153593, -0.192527,  0.006398}}
### backward computation ...
input_grad{{-0.128356,  0.112093}}
